In [ ]:
#--------------------( Importar bibliotecas )--------------------#

import pandas as pd
import pyodbc
import pymssql
import datetime
import smtplib
from smtplib import SMTP
import json
import re
import win32com.client as win32
import schedule
import time
import numpy as np
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.filterwarnings("ignore", category=SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="pandas")

In [ ]:
##--------------------( Configurar o modo como eu vejo as tabelas )--------------------#

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
def conectar_sql():
    server = 'tms-drp-oficial.database.windows.net'
    database = 'tms'
    user = 'appTableau'
    password = 'Tjd82#jks2'

    tentativas = 3

    for tentativa in range(1, tentativas + 1):
        try:
            conexao = pymssql.connect(server=server, database=database, user=user, password=password)
            print(f"Conexão bem-sucedida na tentativa {tentativa}")
            return conexao
        except Exception as e:
            print(f"Tentativa {tentativa} falhou. Erro: {e}")
            if tentativa == tentativas:
                raise


In [ ]:
def gerar_query():
        
    # Query - Aqui pode modificar se precisar
    query = """
    SELECT DISTINCT
    OSV_ID 'Ordem Serviço',
    TEO_NOME 'Status ação nome',
    TEO_ID,
    OSV_DATA_ABERTURA 'Data abertura da OS',
    OSV_DATA_CONCLUSAO 'Conclusao do serviço',
    EOS_DATA_INICIO 'Data Inicio',
    EOS_DATA_FIM 'Data Fim',
    SOS_NOME 'Status da OS',
    MSP_SLA_LIMITE_NEGOCIADO,
    (EOS_DATA_FIM - EOS_DATA_INICIO) tempoDuracao,
    (SELECT DATEDIFF (MINUTE, EOS_DATA_INICIO, EOS_DATA_FIM)) duracaoMinutos,
    (SELECT DATEADD (MINUTE, MSP_SLA_LIMITE_NEGOCIADO, EOS_DATA_INICIO)) tempoFimPrevia,
    --FORMAT(SWITCHOFFSET(GETDATE(), '-03:00'), 'yyyy-MM-dd HH:mm:ss') dataatual
    TSE_NOME 'tipoServiço',
    OSL_MUNICIPIO Cidade,
    OSL_UF Estado,
    TEV_SEG_ID

    FROM ORDEM_SERVICO

    LEFT JOIN ETAPA_ORDEM_SERVICO ON EOS_OSV_ID = OSV_ID
    LEFT JOIN TIPO_ETAPA_ORDEM_SERVICO ON TEO_ID = EOS_TEO_ID
    LEFT JOIN STATUS_ORDEM_SERVICO ON SOS_ID = OSV_SOS_ID
    JOIN MISSAO ON MSO_OSV_ID = OSV_ID
    JOIN MISSAO_PRESTADOR ON MSP_MSO_ID = MSO_ID
    JOIN ACIONAMENTO_PRESTADOR ON APR_MSP_MSO_ID = MSO_ID
    JOIN PRESTADOR ON APR_PRS_ID = PRS_ID
    LEFT JOIN TIPO_SERVICO ON TSE_ID = OSV_TSE_ID
    LEFT JOIN ORDEM_SERVICO_LOCALIDADE ON OSL_OSV_ID = OSV_ID
    LEFT JOIN ASSISTENCIA ON ASI_ID = OSV_ASI_ID
    LEFT JOIN TIPO_EVENTO ON TEV_ID = ASI_TEV_ID

    WHERE CONVERT(DATE, OSV_DATA_ABERTURA) = CONVERT(DATE, GETDATE())
    AND SOS_NOME <> 'Cancelado' 
    AND SOS_NOME <>  'Agendado' 
    AND SOS_NOME <> 'Concluido' 
    AND SOS_NOME <> 'Realizando entrega'
    AND TEO_ID <> 4
    AND TEO_ID <> 2
    AND OSV_DATA_AGENDAMENTO IS NULL
    AND TEV_SEG_ID = 2
    AND OSL_NUMERO_SEQUENCIA = 1

    ORDER BY OSV_ID, TEO_ID
    """

    # Fazer a conexão e olhar os dados
    conexao = conectar_sql()
    df = pd.read_sql_query(query, conexao)
    df1 = df

    conexao.close()
    
    return df, df1

In [ ]:
df = gerar_query()

In [ ]:
tabela = pd.DataFrame(df[0])

In [ ]:
# Query - Aqui pode modificar se precisar
query = """
SELECT DISTINCT
OSV_ID 'Ordem Serviço',
TEO_NOME 'Status ação nome',
TEO_ID,
OSV_DATA_ABERTURA 'Data abertura da OS',
OSV_DATA_CONCLUSAO 'Conclusao do serviço',
EOS_DATA_INICIO 'Data Inicio',
EOS_DATA_FIM 'Data Fim',
SOS_NOME 'Status da OS',
MSP_SLA_LIMITE_NEGOCIADO,
(EOS_DATA_FIM - EOS_DATA_INICIO) tempoDuracao,
(SELECT DATEDIFF (MINUTE, EOS_DATA_INICIO, EOS_DATA_FIM)) duracaoMinutos,
(SELECT DATEADD (MINUTE, MSP_SLA_LIMITE_NEGOCIADO, EOS_DATA_INICIO)) tempoFimPrevia,
--FORMAT(SWITCHOFFSET(GETDATE(), '-03:00'), 'yyyy-MM-dd HH:mm:ss') dataatual
TSE_NOME 'tipoServiço',
OSL_MUNICIPIO Cidade,
OSL_UF Estado,
TEV_SEG_ID

FROM ORDEM_SERVICO

LEFT JOIN ETAPA_ORDEM_SERVICO ON EOS_OSV_ID = OSV_ID
LEFT JOIN TIPO_ETAPA_ORDEM_SERVICO ON TEO_ID = EOS_TEO_ID
LEFT JOIN STATUS_ORDEM_SERVICO ON SOS_ID = OSV_SOS_ID
JOIN MISSAO ON MSO_OSV_ID = OSV_ID
JOIN MISSAO_PRESTADOR ON MSP_MSO_ID = MSO_ID
JOIN ACIONAMENTO_PRESTADOR ON APR_MSP_MSO_ID = MSO_ID
JOIN PRESTADOR ON APR_PRS_ID = PRS_ID
LEFT JOIN TIPO_SERVICO ON TSE_ID = OSV_TSE_ID
LEFT JOIN ORDEM_SERVICO_LOCALIDADE ON OSL_OSV_ID = OSV_ID
LEFT JOIN ASSISTENCIA ON ASI_ID = OSV_ASI_ID
LEFT JOIN TIPO_EVENTO ON TEV_ID = ASI_TEV_ID

WHERE CONVERT(DATE, OSV_DATA_ABERTURA) = CONVERT(DATE, GETDATE())
AND SOS_NOME <> 'Cancelado' 
AND SOS_NOME <>  'Agendado' 
AND SOS_NOME <> 'Concluido' 
AND SOS_NOME <> 'Realizando entrega'
AND TEO_ID <> 4
AND TEO_ID <> 2
AND OSV_DATA_AGENDAMENTO IS NULL
AND TEV_SEG_ID = 2
AND OSL_NUMERO_SEQUENCIA = 1

ORDER BY OSV_ID, TEO_ID
"""

# Fazer a conexão e olhar os dados
conexao = conectar_sql()
df = pd.read_sql_query(query, conexao)
df1 = df

conexao.close()

In [ ]:
df

In [ ]:
def tempoChegada(df):

    df['Hora atual'] = datetime.datetime.now()

        # Criar coluna resultado sendo ele como tempo que finaliza a prévia que foi informada ao cliente
    filtro = (df['Status ação nome'] == 'Chegada') & (df['Data Fim'].isnull())
    df['ResultadoTempoChegada'] = ''
    df.loc[filtro, 'ResultadoTempoChegada'] = (df.loc[filtro, 'Hora atual'].apply(lambda x: x.timestamp()) - df.loc[filtro, 'tempoFimPrevia'].apply(lambda x: x.timestamp())) / 60

    # Formatar todos os campos de data em um único modelo
    df['Data abertura da OS'] = pd.to_datetime(df['Data abertura da OS'])
    df['Data Inicio'] = pd.to_datetime(df['Data Inicio'])
    df['Data Fim'] = pd.to_datetime(df['Data Fim'], errors='coerce')
    df['Hora atual'] = pd.to_datetime(df['Hora atual'])
    df['tempoDuracao'] = pd.to_datetime(df['tempoDuracao'])
    df['tempoFimPrevia'] = pd.to_datetime(df['tempoFimPrevia'])

    # Criar tempo de Experiência até o momento
    df['tempoExperienciaAgora'] = df['Hora atual'] - df['Data abertura da OS']

    # Tranformar em modelo datetime
    tempo_timedelta = pd.to_timedelta(df['tempoExperienciaAgora'])
    data_referencia = pd.Timestamp.now().normalize()
    tempo_datetime = data_referencia + tempo_timedelta

    # Criar um campo com tempo de atendimento ultrapassado, ou seja, vai mostrar em minutos, quanto tempos estamos a partir do final do acionamento
    x = df['ResultadoTempoChegada']
    def convert_to_hms(x):
        if x:
            hours = int(x // 3600)
            minutes = int((x % 3600) // 60)
            seconds = int(x % 60)
            return f"{hours:02d}:{minutes:02d}:{seconds:02d}"
        else:
            return ""

    # Lançar informações nos campos que estão em branco, pois sem dados, não consigo fazer contas
    df['ResultadoTempoChegada'] = pd.to_numeric(df['ResultadoTempoChegada'], errors='coerce')

    # Criar o campo status que vai mostrar se estamos dentro da prévia ou fora
    df['Status'] = df['ResultadoTempoChegada'].apply(lambda x: 'Fora da prévia' if x > 0 else 'Finalizado')

    def formatar_tempo(tempo_minutos):
        horas = tempo_minutos // 60
        minutos = tempo_minutos % 60
        segundos = int((tempo_minutos - int(tempo_minutos)) * 60)
        return '{:02d}:{:02d}:{:02d}'.format(int(horas), int(minutos), segundos)

    # Transformar o campo resultado que estava em minutos no modelo de dia, hora, minutos e segundos
    import numpy as np
    df['tempoForaPrevia'] = np.where(df['ResultadoTempoChegada'] > 0, df['ResultadoTempoChegada'].apply(lambda x: str(pd.to_timedelta(x, unit='m'))), 'Dentro da prévia')

    # Tranformar a coluna tempoForaPrevia em datetime, conforme os demais campos.
    df['tempoForaPrevia'] = df['tempoForaPrevia'].apply(lambda x: pd.Timestamp.now().normalize() + x if isinstance(x, pd.Timedelta) else x)

    ## Trocar valores em branco, com não finalizado
    df['Data Fim'].fillna(value='Em andamento', inplace=True)

    ## Desejo visualizar os serviços que estão em andamento. Vou excluir todos que já foram finalizados
    tabela = df.loc[df['Data Fim'] == 'Em andamento']

    # Serviços com status somente Chegada
    tabelaChegada = tabela.loc[tabela['Status ação nome'] == 'Chegada']

    
    # Tabela Chegada
    tabelaChegadaFinal = tabelaChegada[['Ordem Serviço','Data Inicio','MSP_SLA_LIMITE_NEGOCIADO','tempoFimPrevia','Status','duracaoMinutos','tempoExperienciaAgora', 'Hora atual','tipoServiço', 'Cidade', 'Estado']]

    tabelaChegadaFinal = tabelaChegadaFinal.loc[tabela['Status'] == 'Fora da prévia']

    tabelaChegadaFinal['tempoExcedido'] = tabelaChegadaFinal['Hora atual'] - tabelaChegadaFinal['tempoFimPrevia']

    # Tranformar em modelo datetime - Tempo fora da prévia
    tempo_timedelta = pd.to_timedelta(tabelaChegadaFinal['tempoExcedido'])
    data_referencia = pd.Timestamp.now().normalize()
    tempo_datetime = data_referencia + tempo_timedelta

    # Criar tempo excedido depois da prévia que o prestador informou e deixar como número inteiro (minutos) 
    tabelaChegadaFinal['tempoExcedido'] = round(tabelaChegadaFinal['tempoExcedido'].dt.total_seconds() / 60).astype(int)


    # Renomear a tabela do SLA
    tabelaChegadaFinal = tabelaChegadaFinal.rename(columns={'MSP_SLA_LIMITE_NEGOCIADO': 'SLA'})

    # Criar uma coluna somente com a hora do inicio do acionamento
    tabelaChegadaFinal['Hora Inicio'] = tabelaChegadaFinal['Data Inicio']

    tabelaChegadaFinal['ultimaAtualização'] = tabelaChegadaFinal['Hora atual']

    # Prazo para finalizar a prévia informada pelo prestador
    tabelaChegadaFinal['ultimaAtualização'] = pd.to_datetime(tabelaChegadaFinal['ultimaAtualização'])

    # Tranformei o tempo exedido que estava em minutos para visualização de horas
    tabelaChegadaFinal['tempoForaDaPrévia'] = pd.to_datetime(tabelaChegadaFinal['tempoExcedido'], unit='m').dt.strftime('%H:%M:%S')

    ## Transformar todas as colunas que contenha data, hora e modelo em somente hora

    # Data Inicio
    tabelaChegadaFinal['Data Inicio'] = pd.to_datetime(tabelaChegadaFinal['Data Inicio']) 

    lista = []
    for i in tabelaChegadaFinal['Data Inicio']:
        x = i.strftime("%d/%m")
        lista.append(x)

    tabelaChegadaFinal['Data Inicio'] = lista

    # Hora Inicio
    tabelaChegadaFinal['Hora Inicio'] = pd.to_datetime(tabelaChegadaFinal['Hora Inicio']) 

    lista = []
    for i in tabelaChegadaFinal['Hora Inicio']:
        x = i.strftime("%H:%M:%S")
        lista.append(x)

    tabelaChegadaFinal['Hora Inicio'] = lista

    # Prazo para finalizar a prévia informada pelo prestador
    tabelaChegadaFinal['tempoFimPrevia'] = pd.to_datetime(tabelaChegadaFinal['tempoFimPrevia']) 

    lista = []
    for i in tabelaChegadaFinal['tempoFimPrevia']:
        x = i.strftime("%H:%M:%S")
        lista.append(x)

    tabelaChegadaFinal['tempoFimPrevia'] = lista

    lista = []
    for i in tabelaChegadaFinal['ultimaAtualização']:
        x = i.strftime("%H:%M:%S")
        lista.append(x)

    tabelaChegadaFinal['ultimaAtualização'] = lista

    # Tranformar a coluna tempo de experiência em datetime
    tabelaChegadaFinal['tempoExperienciaAgora'] = tabelaChegadaFinal['tempoExperienciaAgora'].apply(lambda x: pd.Timestamp.now().normalize() + x if isinstance(x, pd.Timedelta) else x)

    # Converter para somente hora
    lista = []
    for i in tabelaChegadaFinal['tempoExperienciaAgora']:
        x = i.strftime("%H:%M:%S")
        lista.append(x)

    tabelaChegadaFinal['tempoExperienciaAgora'] = lista

    # Serviços de 30 à 60 acima do tempo de chegada - Avisar o Coordenador
    filtro4 = (tabelaChegadaFinal['tempoExcedido'] > 30) & (tabelaChegadaFinal['tempoExcedido'] <= 60)
    tabelaChegada1 = tabelaChegadaFinal.loc[filtro4]
    tabelaChegada1 = tabelaChegada1[['Ordem Serviço','Data Inicio','Hora Inicio','SLA', 'tempoFimPrevia','ultimaAtualização','tempoForaDaPrévia','tempoExperienciaAgora','tipoServiço', 'Cidade', 'Estado']]

    # Serviços de 60 à 90 minutos em acionamento - Avisar o Gerente
    filtro5 = (tabelaChegadaFinal['tempoExcedido'] > 60) & (tabelaChegadaFinal['tempoExcedido'] <= 90)
    tabelaChegada2 = tabelaChegadaFinal.loc[filtro5]
    tabelaChegada2 = tabelaChegada2[['Ordem Serviço','Data Inicio','Hora Inicio','SLA', 'tempoFimPrevia','ultimaAtualização','tempoForaDaPrévia','tempoExperienciaAgora','tipoServiço', 'Cidade', 'Estado']]

    # Serviços acima de 90 minutos - Avisar o Superintendente
    filtro6 = (tabelaChegadaFinal['tempoExcedido'] > 90)
    tabelaChegada3 = tabelaChegadaFinal.loc[filtro6]
    tabelaChegada3 = tabelaChegada3[['Ordem Serviço','Data Inicio','Hora Inicio','SLA', 'tempoFimPrevia','ultimaAtualização','tempoForaDaPrévia','tempoExperienciaAgora','tipoServiço', 'Cidade', 'Estado']]

    # Aqui vou montar uma tabela que mostra a quantidade de serviços por faixa de tempo em momento de acionamento

    # Crie um dicionário com as opções e os resultados
    resultadoChegada = {
        'Entre 30 e 60 minutos': tabelaChegada1['Ordem Serviço'].count(),
        'Entre 60 e 90 minutos': tabelaChegada2['Ordem Serviço'].count(),
        'Acima de 90 minutos': tabelaChegada3['Ordem Serviço'].count()
    }

    # Crie a tabela a partir do dicionário
    resultadoChegada = pd.DataFrame.from_dict(resultadoChegada, orient='index', columns=['Quantidade'])

    # Inserir nome na coluna Index
    resultadoChegada.rename_axis('Range',axis=0, inplace = True)

    return resultadoChegada, tabelaChegada1, tabelaChegada2, tabelaChegada3



In [ ]:
df = gerar_query()

In [ ]:
resultadoChegada, tabelaChegada1, tabelaChegada2, tabelaChegada3 = tempoChegada(df)